In [2]:
import numpy as np
import os
from tqdm import tqdm

import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import torchvision.transforms.transforms as TF

import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

from MPRNet import MPRNet
import utils

%load_ext autoreload
%autoreload 2
mps_device = torch.device("mps")
doc_dir = "../../publaynet/"
result_dir = "../results/"
utils.mkdir(result_dir)

In [8]:
model_restoration = MPRNet()

In [9]:
weights_ = "./pretrained_models/model_deblurring.pth"
utils.load_checkpoint(model_restoration, weights_)

In [10]:
model_restoration.to(mps_device)
# model_restoration = nn.DataParallel(model_restoration)
model_restoration.eval()

MPRNet(
  (shallow_feat1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): CAB(
      (CA): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): ReLU(inplace=True)
          (2): Conv2d(24, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): Sigmoid()
        )
      )
      (body): Sequential(
        (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): PReLU(num_parameters=1)
        (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
  )
  (shallow_feat2): Sequential(
    (0): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): CAB(
      (CA): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(96, 

In [11]:
from customDataSet import CustomDataset

In [12]:
data = CustomDataset(doc_dir,(200,200),False)
dataloader = DataLoader(dataset=data, batch_size=1, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)
original_dir = os.path.join(result_dir,"original")
blurred_dir = os.path.join(result_dir,"blurred")
restored_dir = os.path.join(result_dir,"restored")

size 47958


In [ ]:
with torch.no_grad():
    psnr_val_rgb = []
    topil = TF.ToPILImage()
    for ii, data_test in enumerate(tqdm(dataloader), 0):
        # torch.cuda.ipc_collect()
        # torch.cuda.empty_cache()

        blurr = data_test[0].to(mps_device)
        gt = data_test[1].cpu().detach()
        filenames = data_test[2]

        restored = model_restoration(blurr)
        restored = torch.clamp(restored[0],0,1)

        restored = restored.cpu().detach()
        blurr = blurr.cpu().detach()

        for batch in range(len(restored)):
            restored_img = restored[batch]
            psnr_val_rgb.append(utils.torchPSNR(restored_img, gt[batch]))
            utils.save_img((os.path.join(restored_dir, filenames[batch]+'.png')), topil(restored_img))
            utils.save_img((os.path.join(original_dir, filenames[batch]+'.png')), topil(gt[batch]))
            utils.save_img((os.path.join(blurred_dir, filenames[batch]+'.png')), topil(blurr[batch]))

In [ ]:
y,x,name = data[400]
topil = TF.ToPILImage()
y,x = topil(y),topil(x)

fig,(ax1,ax2) = plt.subplots(1,2, figsize=(10,10))
ax1.imshow(y)
ax2.imshow(x)

In [7]:
from data_RGB import get_test_data

dataxx = get_test_data(os.path.join(doc_dir,"train"),img_options=None)
dataloaderxx = DataLoader(dataset=dataxx, batch_size=1, shuffle=False, num_workers=0, drop_last=False, pin_memory=True)

In [21]:
with torch.no_grad():
    psnr_val_rgb = []
    topil = TF.ToPILImage()
    for ii, data_test in enumerate(tqdm(dataloaderxx), 0):
        # torch.cuda.ipc_collect()
        # torch.cuda.empty_cache()

        blurr = data_test[0]
        blurr = blurr[:,:,200:400,200:400].to(mps_device)
        filenames = data_test[1]

        restored = model_restoration(blurr)
        # restored = torch.clamp(restored[0],0,1)

        break

  0%|          | 0/47958 [00:00<?, ?it/s]
